# **Các bước chính thực hiện thuật toán Actor Critic trong code:**
# 1. Khởi tạo môi trường và các tham số cần thiết.
# 2. Trong mỗi episode, Actor chọn hành động dựa trên xác suất, Critic đánh giá hành động thông qua giá trị trạng thái.
# 3. Lưu trữ phần thưởng và thông tin cần thiết trong mỗi bước.
# 4. Khi episode kết thúc, tính giá trị hồi quy (returns) từ phần thưởng nhận được.
# 5. Tính toán lợi thế (advantage) để cải thiện Actor, giảm mất mát của Critic.
# 6. Cập nhật mô hình (Actor và Critic) bằng cách sử dụng gradient descent.
# 7. Lặp lại các bước trên cho đến khi đạt điều kiện dừng.

In [ ]:
# Thêm thư viện
import os
os.environ["KERAS_BACKEND"] = "tensorflow" # Thiết lập backend sử dụng TensorFlow
import gym
import numpy as np
import keras
from keras import ops
from keras import layers
import tensorflow as tf

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


In [ ]:
# **Cấu hình tham số**
seed = 42  # Hạt giống ngẫu nhiên để tái lập kết quả
gamma = 0.99  # Hệ số giảm giá (discount factor) cho giá trị hồi quy
max_steps_per_episode = 500  # Số bước tối đa trong một episode
env = gym.make("CartPole-v1")  # Tạo môi trường CartPole
env.seed(seed)  # Đặt hạt giống cho môi trường
eps = np.finfo(np.float32).eps.item()  # Một giá trị epsilon nhỏ để tránh lỗi chia cho 0

# **Xác định kiến trúc mạng**
num_inputs = 4  # Số lượng đầu vào từ môi trường (4 trạng thái: vị trí, góc, vận tốc, tốc độ góc)
num_actions = 2  # Số lượng hành động (trái hoặc phải)
num_hidden = 128  # Số lượng nơ-ron trong lớp ẩn

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


In [ ]:
# **Xây dựng mạng Actor-Critic**
inputs = layers.Input(shape=(num_inputs,))  # Định nghĩa đầu vào (state)
common = layers.Dense(num_hidden, activation="relu")(inputs)  # Lớp ẩn chung với 128 nơ-ron
action = layers.Dense(num_actions, activation="softmax")(common)  # Lớp Actor dự đoán xác suất các hành động
critic = layers.Dense(1)(common)  # Lớp Critic dự đoán giá trị trạng thái

model = keras.Model(inputs=inputs, outputs=[action, critic])  # Kết hợp Actor và Critic thành một mô hình

# **Tối ưu hóa và hàm mất mát**
optimizer = keras.optimizers.Adam(learning_rate=0.01)  # Trình tối ưu hóa Adam với tốc độ học 0.01
huber_loss = keras.losses.Huber()  # Hàm mất mát Huber (giảm ảnh hưởng của outliers)

In [ ]:
# **Biến lưu lịch sử và thiết lập ban đầu**
action_probs_history = []  # Lưu log xác suất hành động đã chọn
critic_value_history = []  # Lưu giá trị Critic
rewards_history = []  # Lưu phần thưởng trong tập
running_reward = 0  # Phần thưởng trung bình động
episode_count = 0  # Đếm số episode đã hoàn thành
max_episodes = 1000  # Giới hạn số episode để huấn luyện

In [ ]:
# **Vòng lặp huấn luyện**
while episode_count < max_episodes:
    state = env.reset()  # Reset môi trường, lấy trạng thái ban đầu
    episode_reward = 0  # Tổng phần thưởng của episode

    with tf.GradientTape() as tape:  # Bắt đầu theo dõi các phép tính để tính gradient
        for timestep in range(1, max_steps_per_episode + 1):
            state = ops.convert_to_tensor(state)  # Chuyển trạng thái thành tensor
            state = ops.expand_dims(state, 0)  # Thêm chiều batch

            action_probs, critic_value = model(state)  # Mô hình dự đoán xác suất hành động và giá trị Critic
            critic_value_history.append(critic_value[0, 0])  # Lưu giá trị Critic

            action = np.random.choice(num_actions, p=np.squeeze(action_probs))  # Chọn hành động dựa trên xác suất
            action_probs_history.append(ops.log(action_probs[0, action]))  # Lưu log xác suất của hành động đã chọn

            state, reward, done, _ = env.step(action)  # Áp dụng hành động trong môi trường
            rewards_history.append(reward)  # Lưu phần thưởng
            episode_reward += reward  # Cộng dồn phần thưởng

            if done:  # Nếu môi trường báo kết thúc
                break

        # **Tính giá trị hồi quy (returns)**
        returns = []  # Danh sách lưu giá trị hồi quy
        discounted_sum = 0  # Khởi tạo giá trị hồi quy
        for r in rewards_history[::-1]:  # Duyệt ngược danh sách phần thưởng
            discounted_sum = r + gamma * discounted_sum  # Áp dụng công thức giảm giá
            returns.insert(0, discounted_sum)  # Lưu giá trị hồi quy

        returns = np.array(returns)  # Chuyển sang numpy array
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)  # Chuẩn hóa giá trị
        returns = returns.tolist()  # Chuyển lại thành danh sách

        # **Tính toán mất mát**
        history = zip(action_probs_history, critic_value_history, returns)  # Gộp các giá trị để tính toán
        actor_losses = []  # Lưu mất mát của Actor
        critic_losses = []  # Lưu mất mát của Critic
        for log_prob, value, ret in history:
            advantage = ret - value  # Tính lợi thế (advantage)
            actor_losses.append(-log_prob * advantage)  # Mất mát Actor dựa trên lợi thế
            critic_losses.append(huber_loss(ops.expand_dims(value, 0), ops.expand_dims(ret, 0)))  # Mất mát Critic

        loss_value = sum(actor_losses) + sum(critic_losses)  # Tổng mất mát

        grads = tape.gradient(loss_value, model.trainable_variables)  # Tính gradient
        optimizer.apply_gradients(zip(grads, model.trainable_variables))  # Cập nhật trọng số

        action_probs_history.clear()  # Xóa lịch sử hành động
        critic_value_history.clear()  # Xóa lịch sử giá trị Critic
        rewards_history.clear()  # Xóa lịch sử phần thưởng

    episode_count += 1  # Tăng số episode
    running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward  # Cập nhật phần thưởng trung bình động

    if episode_count % 10 == 0:  # Log mỗi 10 episode
        template = "Episode: {}, Running Reward: {:.2f}"
        print(template.format(episode_count, running_reward))

    if running_reward > 475:  # Điều kiện dừng khi đạt phần thưởng mục tiêu
        print("Solved at episode {}!".format(episode_count))
        break

if episode_count == max_episodes:  # Nếu đạt giới hạn số episode
    print(f"Reached the maximum episode limit of {max_episodes}. Training stopped.")

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 10, Running Reward: 15.18
Episode: 20, Running Reward: 25.57
Episode: 30, Running Reward: 23.03
Episode: 40, Running Reward: 19.60
Episode: 50, Running Reward: 16.89
Episode: 60, Running Reward: 17.77
Episode: 70, Running Reward: 21.20
Episode: 80, Running Reward: 37.51
Episode: 90, Running Reward: 39.03
Episode: 100, Running Reward: 72.78
Episode: 110, Running Reward: 57.92
Episode: 120, Running Reward: 43.30
Episode: 130, Running Reward: 37.61
Episode: 140, Running Reward: 57.23
Episode: 150, Running Reward: 83.04
Episode: 160, Running Reward: 114.87
Episode: 170, Running Reward: 90.36
Episode: 180, Running Reward: 69.34
Episode: 190, Running Reward: 55.32
Episode: 200, Running Reward: 52.66
Episode: 210, Running Reward: 50.68
Episode: 220, Running Reward: 63.15
Episode: 230, Running Reward: 111.07
Episode: 240, Running Reward: 118.11
Episode: 250, Running Reward: 119.31
Episode: 260, Running Reward: 120.00
Episode: 270, Running Reward: 120.26
Episode: 280, Running Reward: 1

**Giải thích thêm:**
- Actor dự đoán hành động tối ưu dựa trên xác suất.
- Critic giúp đánh giá hành động bằng cách tính giá trị trạng thái.
- Gradient loss từ cả Actor và Critic giúp cập nhật mạng để cải thiện hiệu suất qua từng episode.